In [ ]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import warnings
import numpy as np
from custom_libraries.load_data_for_label import load_data_for_label
from custom_libraries.merge_data import merge_data
from pycaret.classification import *
from custom_libraries.find_peaks import find_peaks_cust

In [ ]:
pip install pycaret

# Load Test Data

In [ ]:
folder_path = "LABELED"
os.makedirs(folder_path, exist_ok=True)
file_names = []
for name in os.listdir(folder_path):
    file_path = f"{folder_path}/{name}"
    file_names.append(file_path)
#Ahora tengo que especificar mis features 
signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
data = load_data_for_label(filelist=file_names[0:2], signals= signals)
df= pd.concat(data)

In [ ]:
df.shape

In [ ]:
#X = df.drop(['peaks', 'first_sample','last_sample', 'first_sample_closest_peak', 
 #            'last_sample_closest_peak','exercise','exercising_periods'], axis=1)
X = df.copy()
X.columns

# Auxiliary Functions

In [ ]:
def prepare_data(chunk_df):
    signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
    stats_dict = {}
    series = chunk_df[signals]
    for column in series.columns:
        mean = series[column].mean()
        std = series[column].std()
        median = series[column].median()
    
        stats_dict[f"{column}_mean"] = mean
        stats_dict[f"{column}_std"] = std
        stats_dict[f"{column}_median"] = median
        
    return pd.DataFrame.from_dict(stats_dict, orient="index").T

# Load Model

In [ ]:
# Load the model
pipeline = load_model(model_name="squats_traditional")

In [ ]:
ventana = prepare_data(X.iloc[0:51])
prediction = predict_model(pipeline, ventana, raw_score=True)
prediction.prediction_label.iloc[0]

# Check Data

In [ ]:
nuevo_df_predicciones = pd.DataFrame()
nuevo_df_list = []  # Lista para almacenar las filas

for idx, fila in df.iterrows():
    warnings.filterwarnings('ignore')
    nuevo_df_list.append(fila.copy())  # Añadir la fila a la lista
    
    if pd.notna(fila['peaks']):
        nuevo_df = pd.DataFrame(nuevo_df_list)  # Convertir la lista en DataFrame
        X1 = prepare_data(nuevo_df)
        prediction = predict_model(pipeline, X1, raw_score=True)
        y_pred = prediction.prediction_label.iloc[0]
        X1['start'] = nuevo_df.index[0]
        X1['end'] = nuevo_df.index[-1]
        X1['prediccion'] = y_pred
        nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)
        
        # Limpiar la lista para la próxima ventana
        nuevo_df_list = [fila.copy()]

In [ ]:
nuevo_df_predicciones.shape

In [ ]:
nuevo_df_predicciones.head()

In [ ]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)

In [ ]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 'SQUAT'][['start', 'end', 'prediccion']]
squats_detection

In [ ]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == ('NO_EXERCISE'
                                                                              ''
                                                                              '')][['start', 'end', 'prediccion']]
squats_detection